# L5: Information extraction

Information extraction (IE) is the task of identifying named entities and semantic relations between these entities in text data. In this lab we will focus on two sub-tasks in IE, **named entity recognition** (identifying mentions of entities) and **entity linking** (matching these mentions to entities in a knowledge base).


Students:

Weng Hang Wong (wenwo535)

Zuxiang Li(zuxli371)

We start by loading spaCy:

In [74]:
import spacy
from spacy.gold import GoldParse

nlp = spacy.load('en_core_web_sm')


The data that we will be using has been tokenized following the conventions of the [Penn Treebank](ftp://ftp.cis.upenn.edu/pub/treebank/public_html/tokenization.html), and we need to prevent spaCy from using its own tokenizer on top of this. We therefore override spaCy&rsquo;s tokenizer with one that simply splits on space.

In [75]:
from spacy.tokens import Doc

class WhitespaceTokenizer(object):
    def __init__(self, vocab):
        self.vocab = vocab

    def __call__(self, text):
        return Doc(self.vocab, words=text.split(' '))

nlp.tokenizer = WhitespaceTokenizer(nlp.vocab)

## Data set

The main data set for this lab is a collection of news wire articles in which mentions of named entities have been annotated with page names from the [English Wikipedia](https://en.wikipedia.org/wiki/). The next code cell loads the training and the development parts of the data into Pandas data frames.

In [76]:
import bz2
import csv
import pandas as pd

with bz2.open('ner-train.tsv.bz2', 'rt',encoding="utf-8") as source:
    df_train = pd.read_csv(source, sep='\t', quoting=csv.QUOTE_NONE,encoding='utf-8')

with bz2.open('ner-dev.tsv.bz2', 'rt',encoding="utf-8") as source:
    df_dev = pd.read_csv(source, sep='\t', quoting=csv.QUOTE_NONE,encoding='utf-8')

Each row in these two data frames corresponds to one mention of a named entity and has five columns:

1. a unique identifier for the sentence containing the entity mention
2. the pre-tokenized sentence, with tokens separated by spaces
3. the start position of the token span containing the entity mention
4. the end position of the token span (exclusive, as in Python list indexing)
5. the entity label; either a Wikipedia page name or the generic label `--NME--`

The following cell prints the first five samples from the training data:

In [77]:
df_train.head()

,sentence_id,sentence,beg,end,label
0,0000-000,EU rejects German call to boycott British lamb .,0,1,--NME--
1,0000-000,EU rejects German call to boycott British lamb .,2,3,Germany
2,0000-000,EU rejects German call to boycott British lamb .,6,7,United_Kingdom
3,0000-001,Peter Blackburn,0,2,--NME--
4,0000-002,BRUSSELS 1996-08-22,0,1,Brussels


In this sample, we see that the first sentence is annotated with three entity mentions:

* the span 0–1 &lsquo;EU&rsquo; is annotated as a mention but only labelled with the generic `--NME--`
* the span 2–3 &lsquo;German&rsquo; is annotated with the page [Germany](http://en.wikipedia.org/wiki/Germany)
* the span 6–7 &lsquo;British&rsquo; is annotated with the page [United_Kingdom](http://en.wikipedia.org/wiki/United_Kingdom)

## Problem 1: Evaluation measures

To warm up, we ask you to write code to print the three measures that you will be using for evaluation:

In [78]:
def evaluation_report(gold, pred):
    """Print precision, recall, and F1 score.

    Args:
        gold: The set with the gold-standard values.
        pred: The set with the predicted values.
    
    Returns:
        Nothing, but prints the precision, recall, and F1 values computed
        based on the specified sets.
    """
    # TODO: Replace the next line with your own code
    precision=len(gold.intersection(pred))/len(pred)*100
    recall=len(gold.intersection(pred))/len(gold)*100
    F1=2*(recall*precision)/(precision+recall)
    print(precision,recall,F1)

To test your code, you can run the following cell:

In [79]:
evaluation_report(set(range(3)), set(range(5)))

60.0 100.0 75.0


This should give you a precision of 60%, a recall of 100%, and an F1-value of 75%.

## Problem 2: Span recognition

One of the first tasks that an information extraction system has to solve is to locate and classify (mentions of) named entities, such as persons and organizations. Here we will tackle the simpler task of recognizing **spans** of tokens that contain an entity mention, without the actual entity label.

The English language model in spaCy features a full-fledged [named entity recognizer](https://spacy.io/usage/linguistic-features#named-entities) that identifies a variety of entities, and can be updated with new entity types by the user. Your task in this problem is to evaluate the performance of this component when predicting entity spans in the development data.

Start by implementing a generator function that yields the gold-standard spans in a given data frame.

**Hint:** The Pandas method [`itertuples()`](https://pandas.pydata.org/pandas-docs/version/0.17.0/generated/pandas.DataFrame.itertuples.html) is useful when iterating over the rows in a DataFrame.

In [80]:
def gold_spans(df):
    """Yield the gold-standard mention spans in a data frame.

    Args:
        df: A data frame.

    Yields:
        The gold-standard mention spans in the specified data frame as
        triples consisting of the sentence id, start position, and end
        position of each span.
    """
    # TODO: Replace the next line with your own code
    #yield
    for x in df.itertuples():
        yield x[1],x[3],x[4]

To test your code, you can count the spans yielded by your function. When called on the development data, you should get a total of 5,917 unique triples. The first triple and the last triple should be

    ('0946-000', 2, 3)
    ('1161-010', 1, 3)  

In [81]:
spans_dev_gold = set(gold_spans(df_dev))
print(len(spans_dev_gold))

5917


Your next task is to write code that calls spaCy to predict the named entities in the development data, and to evaluate the accuracy of these predictions in terms of precision, recall, and F1. Print these scores using the function that you wrote for Problem&nbsp;1.

In [82]:
# TODO: Write code here to run and evalp;uate the spaCy NER on the development data
ent_pred=[]
for x in df_dev.itertuples(index=False,name=None):
    doc = nlp(x[1])
    for ent in doc.ents:
        #print(ent)
        ent_pred.append((x[0],ent.start,ent.end))
        #print(ent.text, ent.start_char, ent.end_char, ent.label_)
print(ent_pred)

 18), ('1152-016', 22, 24), ('1152-016', 25, 26), ('1152-016', 26, 27), ('1152-016', 1, 2), ('1152-016', 2, 3), ('1152-016', 8, 9), ('1152-016', 11, 12), ('1152-016', 14, 15), ('1152-016', 16, 17), ('1152-016', 17, 18), ('1152-016', 22, 24), ('1152-016', 25, 26), ('1152-016', 26, 27), ('1152-016', 1, 2), ('1152-016', 2, 3), ('1152-016', 8, 9), ('1152-016', 11, 12), ('1152-016', 14, 15), ('1152-016', 16, 17), ('1152-016', 17, 18), ('1152-016', 22, 24), ('1152-016', 25, 26), ('1152-016', 26, 27), ('1152-016', 1, 2), ('1152-016', 2, 3), ('1152-016', 8, 9), ('1152-016', 11, 12), ('1152-016', 14, 15), ('1152-016', 16, 17), ('1152-016', 17, 18), ('1152-016', 22, 24), ('1152-016', 25, 26), ('1152-016', 26, 27), ('1152-016', 1, 2), ('1152-016', 2, 3), ('1152-016', 8, 9), ('1152-016', 11, 12), ('1152-016', 14, 15), ('1152-016', 16, 17), ('1152-016', 17, 18), ('1152-016', 22, 24), ('1152-016', 25, 26), ('1152-016', 26, 27), ('1152-017', 3, 4), ('1152-017', 6, 7), ('1152-017', 9, 14), ('1152-017'

In [83]:
spans_dev_pred=set(ent_pred)
print(len(spans_dev_pred))
evaluation_report(spans_dev_gold,spans_dev_pred)

8034
51.53099327856609 69.9678891330066 59.35058418751344


## Problem 3: Error analysis

As you were able to see in Problem&nbsp;2, the span accuracy of the named entity recognizer is far from perfect. In particular, only slightly more than half of the predicted spans are correct according to the gold standard. Your next task is to analyse this result in more detail.

Here is a function that prints the false positives as well as the false negatives spans for a data frame, given a reference set of gold-standard spans and a candidate set of predicted spans.

In [84]:
from collections import defaultdict

def error_report(df, spans_gold, spans_pred):
    false_pos = defaultdict(list)
    for s, b, e in spans_pred - spans_gold:
        false_pos[s].append((b, e))
    false_neg = defaultdict(list)
    for s, b, e in spans_gold - spans_pred:
        false_neg[s].append((b, e))
    for row in df.drop_duplicates('sentence_id').itertuples():
        if row.sentence_id in false_pos or row.sentence_id in false_neg:
            print('Sentence:', row.sentence)
            for b, e in false_pos[row.sentence_id]:
                print('  FP:', ' '.join(row.sentence.split()[b:e]))
            for b, e in false_neg[row.sentence_id]:
                print('  FN:', ' '.join(row.sentence.split()[b:e]))

Use this function to inspect and analyse the errors that the automated prediction makes. Can you see any patterns? Base your analysis on the first 500 rows of the training data. Summarize your observations in a short text.

In [85]:
# TODO: Write code here to do your analysis
ent_pred=[]
print(1111)
for x in df_train.iloc[:500].itertuples():
    doc = nlp(x[2])
    for ent in doc.ents:
        ent_pred.append((x[1],ent.start,ent.end))
spans_train_pred=set(ent_pred)
error_report(df_train.iloc[:500],set(gold_spans(df_train)),spans_train_pred)

tified alarm through " dangerous generalisation . "
  FP: Spanish Farm
  FN: Spanish
Sentence: The EU 's scientific veterinary and multidisciplinary committees are due to re-examine the issue early next month and make recommendations to the senior veterinary officials .
  FP: early next month
Sentence: British farmers denied on Thursday there was any danger to human health from their sheep , but expressed concern that German government advice to consumers to avoid British lamb might influence consumers across Europe .
  FP: Thursday
Sentence: " What we have to be extremely careful of is how other countries are going to take Germany 's lead , " Welsh National Farmers ' Union ( NFU ) chairman John Lloyd Jones said on BBC radio .
  FP: BBC
  FN: BBC radio
Sentence: Bonn has led efforts to protect public health after consumer confidence collapsed in March after a British report suggested humans could contract an illness similar to mad cow disease by eating contaminated beef .
  FP: March
S

*TODO: Write a short text that summarises the errors that you observed*

False Positive is the wrongly predicted entities and we can see from previous result that most of the FPs are numbers, locations, time and date.

Now, use the insights from your error analysis to improve the automated prediction that you implemented in Problem&nbsp;2. While the best way to do this would be to [update spaCy&rsquo;s NER model](https://spacy.io/usage/linguistic-features#updating) using domain-specific training data, for this lab it suffices to write code to post-process the output produced by spaCy. You should be able to improve the F1 score from Problem&nbsp;2 by at last 15 percentage points.

In [86]:
# TODO: Write code here to improve the span prediction from Problem 2

ent_pred=[]
for x in df_dev.itertuples():
    doc = nlp(x[2])
    for ent in doc.ents:
        #print(ent)
        if ent.label_ not in ['DATE', 'TIME', 'PERCENT', 'MONEY', 'QUANTITY', 'ORDINAL', 'CARDINAL']:
            ent_pred.append((x[1],ent.start,ent.end))
print(ent_pred)

50-006', 2, 3), ('1150-006', 12, 16), ('1150-006', 2, 3), ('1150-006', 12, 16), ('1151-000', 6, 7), ('1151-001', 0, 1), ('1151-002', 6, 7), ('1151-006', 20, 21), ('1152-000', 0, 1), ('1152-000', 5, 6), ('1152-000', 7, 8), ('1152-000', 0, 1), ('1152-000', 5, 6), ('1152-000', 7, 8), ('1152-000', 0, 1), ('1152-000', 5, 6), ('1152-000', 7, 8), ('1152-001', 0, 2), ('1152-002', 0, 1), ('1152-003', 3, 4), ('1152-003', 15, 17), ('1152-003', 21, 22), ('1152-003', 24, 25), ('1152-003', 29, 30), ('1152-003', 31, 32), ('1152-003', 3, 4), ('1152-003', 15, 17), ('1152-003', 21, 22), ('1152-003', 24, 25), ('1152-003', 29, 30), ('1152-003', 31, 32), ('1152-003', 3, 4), ('1152-003', 15, 17), ('1152-003', 21, 22), ('1152-003', 24, 25), ('1152-003', 29, 30), ('1152-003', 31, 32), ('1152-003', 3, 4), ('1152-003', 15, 17), ('1152-003', 21, 22), ('1152-003', 24, 25), ('1152-003', 29, 30), ('1152-003', 31, 32), ('1152-003', 3, 4), ('1152-003', 15, 17), ('1152-003', 21, 22), ('1152-003', 24, 25), ('1152-003',

In [87]:
evaluation_report(spans_dev_gold,set(ent_pred))


86.05619146722164 69.88338685144498 77.1311322514456


Show that you achieve the performance goal by reporting the evaluation measures that you implemented in Problem&nbsp;1.

Before going on, we ask you to store the outputs of the improved named entity recognizer on the development data in a new data frame. This new frame should have the same layout as the original data frame for the development data that you loaded above, but should contain the *predicted* start and end positions for each token span, rather than the gold positions. As the `label` of each span, you can use the special value `--NME--`.

In [88]:
# TODO: Write code here to store the predicted spans in a new data frame
#df_dev[df_dev.sentence_id=="0946-002"].iloc[0].sentence
# sentence_id=[x[0] for x in ent_pred]
for i,e in enumerate(ent_pred):
    ent_pred[i]=(e[0],df_dev[df_dev.sentence_id==e[0]].iloc[0].sentence,e[1],e[2],"--NME--")
pred_df=pd.DataFrame(set(ent_pred))

## Problem 4: Entity linking

Now that we have a method for predicting mention spans, we turn to the task of **entity linking**, which amounts to predicting the knowledge base entity that is referenced by a given mention. In our case, for each span we want to predict the Wikipedia page that this mention references.

Start by extending the generator function that you implemented in Problem&nbsp;2 to labelled spans.

In [89]:
def gold_mentions(df):
    """Yield the gold-standard mentions in a data frame.

    Args:
        df: A data frame.

    Yields:
        The gold-standard mention spans in the specified data frame as
        quadruples consisting of the sentence id, start position, end
        position and entity label of each span.
    """
    # TODO: Replace the next line with your own code
    for x in df.itertuples():
        yield x[1],x[3],x[4],x[5]

A naive baseline for entity linking on our data set is to link each mention span to the Wikipedia page name that we get when we join the tokens in the span by underscores, as is standard in Wikipedia page names. Suppose, for example, that a span contains the two tokens

    Jimi Hendrix

The baseline Wikipedia page name for this span would be

    Jimi_Hendrix

Implement this naive baseline and evaluate its performance. Print the evaluation measures that you implemented in Problem&nbsp;1.

<div class="alert alert-warning">
    Here and in the remainder of this lab, you should base your entity predictions on the predicted spans that you computed in Problem&nbsp;3.
</div>

In [90]:
# TODO: Write code here to implement the baseline
naive_pred=[]

# for x in pred_df.iloc[:2].itertuples():
#     doc = nlp(x[2])
#     for ent in doc.ents:
#         tmp_str=[]
#         for i in range(ent.start,ent.end):
#             tmp_str.append(doc[i].text)
        
#         tmp_str="_".join(tmp_str)
#         naive_pred.append((x[1],ent.start,ent.end,tmp_str))
# print(naive_pred)


for x in pred_df.itertuples():
    doc = nlp(x[2])
    ents = [(x[1],e.start,e.end,e.text.replace(" ","_")) for e in doc.ents]
    for e in ents:
        naive_pred.append(e)
print(naive_pred)

, 16, 18, 'Prince_Eugene'), ('1116-006', 20, 21, 'French'), ('1116-006', 31, 32, 'Italy'), ('1046-006', 16, 17, 'ten'), ('1046-006', 19, 21, 'Armando_Alvarez'), ('1046-006', 24, 26, '15_minutes'), ('1116-006', 0, 1, '1706'), ('1116-006', 2, 3, 'French'), ('1116-006', 5, 6, 'Duke'), ('1116-006', 7, 8, 'Orleans'), ('1116-006', 9, 10, 'Turin'), ('1116-006', 14, 15, 'Austrians'), ('1116-006', 16, 18, 'Prince_Eugene'), ('1116-006', 20, 21, 'French'), ('1116-006', 31, 32, 'Italy'), ('1072-012', 5, 6, 'Joubert'), ('1072-012', 11, 14, 'the_25th_minute'), ('1072-012', 18, 20, 'Pieter_Hendriks'), ('1072-012', 24, 25, 'Joubert'), ('1072-012', 31, 33, 'Derek_Bevan'), ('1072-012', 34, 35, 'Van'), ('1072-012', 36, 37, 'Westhuizen'), ('0957-026', 0, 2, 'Andrei_Medvedev'), ('0957-026', 3, 4, 'Ukraine'), ('0957-026', 6, 8, 'Jan_Kroslak'), ('0957-026', 9, 10, 'Slovakia'), ('1072-017', 0, 3, 'Centre_Walter_Little'), ('1072-017', 5, 6, 'Mehrtens'), ('1072-017', 15, 17, 'Justin_Marshall'), ('1058-026', 1, 

In [91]:
evaluation_report(set(gold_mentions(df_dev)),set(naive_pred))

20.426788350556972 25.7224945073517 22.77079593058049


## Problem 5: Extending the training data using the knowledge base

State-of-the-art approaches to entity linking exploit information in knowledge bases. In our case, where Wikipedia is the knowledge base, one particularly useful type of information are links to other Wikipedia pages. In particular, we can interpret the anchor texts (the highlighted texts that you click on) as mentions of the entities (pages) that they link to. This allows us to harvest long lists of mention–entity pairings.

The following cell loads a data frame summarizing anchor texts and page references harvested from the first paragraphs of the English Wikipedia. The data frame also contains all entity mentions in the training data (but not the development or the test data).

In [92]:
with bz2.open('kb.tsv.bz2', 'rt',encoding="utf-8") as source:
    df_kb = pd.read_csv(source, sep='\t', quoting=csv.QUOTE_NONE)

To understand what information is available in this data, the following cell shows the entry for the anchor text `Sweden`.

In [93]:
df_kb.loc[df_kb.mention == 'Sweden']

,mention,entity,prob
17436,Sweden,Sweden,0.985768
17437,Sweden,Sweden_national_football_team,0.014173
17438,Sweden,Sweden_men's_national_ice_hockey_team,0.000059


As you can see, each row of the data frame contains a pair $(m, e)$ of a mention $m$ and an entity $e$, as well as the conditional probability $P(e|m)$ for mention $m$ referring to entity $e$. These probabilities were estimated based on the frequencies of mention–entity pairs in the knowledge base. The example shows that the anchor text &lsquo;Sweden&rsquo; is most often used to refer to the entity [Sweden](http://en.wikipedia.org/wiki/Sweden), but in a few cases also to refer to Sweden&rsquo;s national football and ice hockey teams. Note that references are sorted in decreasing order of probability, so that the most probable pairing come first.

Implement an entity linking method that resolves each mention to the most probable entity in the data frame. If the mention is not included in the data frame, you can predict the generic label `--NME--`. Print the precision, recall, and F1 of your method using the function that you implemented for Problem&nbsp;1.

In [94]:
# TODO: Write code here to implement the "most probable entity" method.
wiki_pred=[]
# for x in pred_df.itertuples():
#     doc = nlp(x[2])
#     for ent in doc.ents:
#         tmp_str=[]
#         for i in range(ent.start,ent.end):
#             tmp_str.append(doc[i].text)
#         tmp_str=" ".join(tmp_str)
#         tmp_df=df_kb.loc[df_kb.mention == tmp_str]
#         if len(tmp_df)!=0:
#             wiki_pred.append((x[1],ent.start,ent.end,tmp_df.iloc[0].entity))
#         else:
#             wiki_pred.append((x[1],ent.start,ent.end,"--NME--"))

for x in pred_df.itertuples():
    doc=nlp(x[2])
    tmp_str=str(doc[x[3]:x[4]])
    tmp_df=df_kb.loc[df_kb.mention==tmp_str]
    if len(tmp_df)!=0:
        new_ent=(x[1],x[3],x[4],tmp_df.iloc[0].entity)
    else:
        new_ent=(x[1],x[3],x[4],"--NME--")
    wiki_pred.append(new_ent)


print(wiki_pred)

'), ('1146-006', 7, 8, 'Saddam_Hussein'), ('1152-004', 49, 52, 'United_States'), ('0946-014', 9, 11, 'Paul_Johnson_(squash_player)'), ('0974-011', 3, 5, 'Muttiah_Muralitharan'), ('1039-011', 29, 30, 'Moro_National_Liberation_Front'), ('1097-001', 0, 2, '--NME--'), ('1132-004', 0, 2, '--NME--'), ('1141-008', 0, 1, 'Guangzhou'), ('1151-000', 6, 7, 'Texas'), ('0984-008', 15, 16, '--NME--'), ('1111-005', 1, 4, '--NME--'), ('1103-020', 17, 18, 'Italy'), ('1144-002', 35, 36, '--NME--'), ('1152-011', 21, 22, 'Baghdad'), ('1160-010', 27, 28, 'HarperCollins'), ('0969-001', 0, 1, 'OGC_Nice'), ('0965-009', 16, 19, '--NME--'), ('0966-057', 1, 3, 'Vladimir_Dubrovshchik'), ('1056-012', 0, 3, '--NME--'), ('0953-003', 7, 8, 'Mexico'), ('0972-036', 9, 10, 'Glenn_McGrath'), ('1043-004', 0, 1, 'Singapore'), ('0957-007', 6, 8, 'Doug_Flach'), ('0965-003', 8, 10, 'Donovan_Bailey'), ('0966-083', 4, 5, '--NME--'), ('1016-007', 22, 25, '--NME--'), ('1031-016', 9, 10, 'China'), ('1053-003', 19, 22, '--NME--'), 

In [95]:
evaluation_report(set(gold_mentions(df_dev)),set(wiki_pred))

66.01456815816857 53.608247422680414 59.1680656593919


## Problem 6: Context-sensitive disambiguation

Consider the entity mention &lsquo;Lincoln&rsquo;. The most probable entity for this mention turns out to be [Lincoln, Nebraska](http://en.wikipedia.org/Lincoln,_Nebraska); but in pages about American history, we would be better off to predict [Abraham Lincoln](http://en.wikipedia.org/Abraham_Lincoln). This suggests that we should try to disambiguate between different entity references based on the textual context on the page from which the mention was taken. Your task in this last problem is to implement this idea.

Set up a dictionary that contains, for each mention $m$ that can refer to more than one entity $e$, a separate Naive Bayes classifier that is trained to predict the correct entity $e$, given the textual context of the mention. As the prior probabilities of the classifier, choose the probabilities $P(e|m)$ that you used in Problem&nbsp;5. To let you estimate the context-specific probabilities, we have compiled a data set with mention contexts:

In [96]:
with bz2.open('contexts.tsv.bz2') as source:
    df_contexts = pd.read_csv(source, sep='\t', quoting=csv.QUOTE_NONE)

This data frame contains, for each ambiguous mention $m$ and each knowledge base entity $e$ to which this mention can refer, up to 100 randomly selected contexts in which $m$ is used to refer to $e$. For this data, a **context** is defined as the 5 tokens to the left and the 5 tokens to the right of the mention. Here are a few examples:

In [97]:
df_contexts.head()

,mention,entity,context
0,1970,UEFA_Champions_League,Cup twice the first in @ and the second in 1983
1,1970,FIFA_World_Cup,America 1975 and during the @ and 1978 World C...
2,1990 World Cup,1990_FIFA_World_Cup,Manolo represented Spain at the @
3,1990 World Cup,1990_FIFA_World_Cup,Hašek represented Czechoslovakia at the @ and ...
4,1990 World Cup,1990_FIFA_World_Cup,renovations in 1989 for the @ The present capa...


Note that, in each context, the position of the mention is indicated by the `@` symbol.

From this data frame, it is easy to select the data that you need to train the classifiers – the contexts and corresponding entities for all mentions. To illustrate this, the following cell shows how to select all contexts that belong to the mention &lsquo;Lincoln&rsquo;:

In [98]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from collections import Counter

classifiers={}
for w in set(df_contexts.mention):
    X=df_contexts.context[df_contexts.mention==w]
    y=df_contexts.entity[df_contexts.mention==w]
 
    c=Counter(y)
    prior = sorted(c.items(), key=lambda pair: (pair[0])) 
    prior= [p[1]/sum(c.values()) for p in prior]
    pipe=Pipeline([("vectorize",CountVectorizer()),("MultinomialNB",MultinomialNB(class_prior=prior))])

    pipe.fit(X,y)
    classifiers[w]=pipe


Implement the context-sensitive disambiguation method and evaluate its performance. Here are some more hints that may help you along the way:

**Hint 1:** The prior probabilities for a Naive Bayes classifier can be specified using the `class_prior` option. You will have to provide the probabilities in the same order as the alphabetically sorted class (entity) names.

**Hint 2:** Not all mentions in the knowledge base are ambiguous, and therefore not all mentions have context data. If a mention has only one possible entity, pick that one. If a mention has no entity at all, predict the `--NME--` label.

In [99]:
# TODO: Write code here to implement the context-sensitive disambiguation method
cs_pred=[]


for x in pred_df.itertuples():
    doc=nlp(x[2])
    tmp_str=str(doc[x[3]:x[4]])
    tmp_df=df_kb.loc[df_kb.mention==tmp_str]
    try:
        cs_pred.append((x[1],x[3],x[4],classifiers[tmp_str].predict([x[2]])[0]))
    except KeyError:
        if len(tmp_df)!=0:
            new_ent=(x[1],x[3],x[4],tmp_df.iloc[0].entity)
            cs_pred.append(new_ent)
        else:
            new_ent=(x[1],x[3],x[4],"--NME--")
            cs_pred.append(new_ent)
print(cs_pred)


# for x in pred_df.itertuples():
#     doc = nlp(x[2])

#     for ent in doc.ents:
#         tmp_str=[]
#         for i in range(ent.start,ent.end):
#             tmp_str.append(doc[i].text)
#         tmp_str=" ".join(tmp_str)
#         tmp_df=df_kb.loc[df_kb.mention == tmp_str]
#         try:
#             cs_pred.append((x[1],ent.start,ent.end,classifiers[tmp_str].predict([x[2]])[0]))
#         except KeyError:
#             if len(tmp_df)!=0:
#                 cs_pred.append((x[1],ent.start,ent.end,tmp_df.iloc[0].entity))
#             else:
#                 cs_pred.append((x[1],ent.start,ent.end,"--NME--"))



# cs_pred=[]
# for x in pred_df.itertuples():
#     doc = nlp(x[2])
#     for ent in doc.ents:
#         tmp_str=[]
#         for i in range(ent.start,ent.end):
#             tmp_str.append(doc[i].text)
#     tmp_str=" ".join(tmp_str)
#     try:
#         cs_pred.append((x[1],ent.start,ent.end,classifiers[tmp_str].predict([x[2]])[0]))
#     except KeyError:
#         cs_pred.append((x[1],ent.start,ent.end,"--NME--"))

# print(cs_pred)

s_(baseball)'), ('0946-006', 0, 1, 'Essex_County_Cricket_Club'), ('1155-011', 3, 4, 'Germany'), ('0957-006', 11, 12, 'Australia_national_cricket_team'), ('1089-001', 2, 3, 'Wisconsin'), ('0966-087', 1, 3, 'Rohan_Robinson'), ('0948-003', 4, 5, 'England_cricket_team'), ('1056-009', 4, 5, 'United_Kingdom'), ('0955-009', 0, 1, '--NME--'), ('1030-040', 1, 2, 'Tokyo'), ('1058-003', 6, 8, '--NME--'), ('1142-005', 17, 18, 'Kurdistan_Democratic_Party'), ('1152-017', 3, 4, 'United_States_Marine_Corps'), ('0984-006', 29, 30, 'Missouri'), ('1056-043', 3, 4, 'France_national_football_team'), ('1058-009', 15, 16, 'Texas'), ('0981-000', 0, 3, '--NME--'), ('1017-002', 19, 21, 'Air_France'), ('0957-012', 0, 2, 'Alexander_Volkov_(tennis)'), ('0957-020', 35, 36, 'Australia_national_cricket_team'), ('1075-002', 0, 1, 'Mauritania_national_football_team'), ('0955-014', 0, 1, 'FC_Seoul'), ('1018-004', 14, 15, 'Berlin'), ('1086-002', 0, 1, 'England_national_football_team'), ('1099-008', 17, 18, 'Duncan_Fergus

In [100]:
evaluation_report(set(gold_mentions(df_dev)),set(cs_pred))


62.70551508844954 50.92107486902147 56.202201081887715


You should expect to see a small (around 1&nbsp;unit) increase in both precision, recall, and F1.

**This was the last lab in the Text Mining course. Congratulations!**

<div class="alert alert-info">
    Please read the section ‘General information’ on the ‘Labs’ page of the course website before submitting this notebook!
</div>